# Boilerplate

Import some useful packages. Note that I set mplstyle as `fast` to increase rendering efficiency.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as colormaps
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import numpy as np
import scipy
from matplotlib.colors import LogNorm
mplstyle.use('fast')

ModuleNotFoundError: No module named 'matplotlib'

Notebook settings for animation:

In [ ]:
animate = True
from cplt3d.generator_funcs import spin_3d_plot

Generate some data for the Gaussian pdf and take some samples:

In [ ]:
# generate the pdf of the Gaussian
N = 32
X,Y,Z = np.linspace(-4,4,N),np.linspace(-4,4,N),np.linspace(-4,4,N)
X,Y,Z = np.meshgrid(X,Y,Z)
X = X.flatten()
Y = Y.flatten()
Z = Z.flatten()
pts = np.array([X,Y,Z]).T
GAUSSIAN = scipy.stats.multivariate_normal.pdf(pts,mean = [0,0,0],cov = [[1,0,0],[0,2,0],[0,0,3]])

In [ ]:
# take some samples from a Gaussian
N = 10000
std = 0.3
np.random.seed(123456)
pts_samples = np.random.multivariate_normal(mean = [0,0,0],cov = [[std,0,0],[0,std,0],[0,0,std]],size = N)
GAUSSIAN_samples = np.ones(len(pts_samples))

In [ ]:
# take some samples from a bimodal distribution
N = 10000
std = 0.3
pos = 1
pts_samples_bi_1 = np.random.multivariate_normal(mean = [pos,pos,pos],cov = [[std,0,0],[0,std,0],[0,0,std]],size = N)
pts_samples_bi_2 = np.random.multivariate_normal(mean = [-pos,-pos,-pos],cov = [[std,0,0],[0,std,0],[0,0,std]],size = N)
pts_samples_bi = list(pts_samples_bi_1) + list(pts_samples_bi_2)
GAUSSIAN_samples_bi = np.ones(len(pts_samples_bi))

# Uniform Plotting

Import `cplt3d`'s generator functions that we are interested in:

In [ ]:
from cplt3d.generator_funcs import uniform_histogram,uniform_nearest_interpolator,uniform_linear_interpolator

Set the number of bins to use in plotting:

In [ ]:
N_bins = 2**4

We can easily plot using `matplotlib` objects. For instance, we can plot a nearest interpolation of the Gaussian:

In [ ]:
# Prepare the figure 
fig = plt.figure(figsize = (3,3))
ax = fig.add_subplot(projection = '3d')

# Prepare the colormap. Note that we use a non-linear alpha to make the center clearer
cmap = colormaps.get_cmap('viridis')
use_cmap = cmap(np.arange(cmap.N))
use_cmap[:,-1] = np.linspace(0,1,cmap.N)**1.7
use_cmap = colors.ListedColormap(use_cmap)

# Plot the function
uniform_nearest_interpolator(ax,pts,GAUSSIAN,bins = N_bins,filled = 0.3,cmap = use_cmap,verbose = True)

# Edit some figure settings and save it
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.zaxis.labelpad=-3.5
fig.savefig('./Images/1_Nearest-Uniform_Gaussian.png',dpi = 300)

Or a linear interpolation of the Gaussian:

In [ ]:
# Prepare the figure 
fig = plt.figure(figsize = (3,3))
ax = fig.add_subplot(projection = '3d')

# Plot the function
uniform_linear_interpolator(ax,pts,GAUSSIAN,bins = N_bins,filled = 0.3,cmap = use_cmap,verbose = False)

# Edit some figure settings and save it
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.zaxis.labelpad=-3.5
fig.savefig('Images/2_Linear-Uniform_Gaussian.png',dpi = 300)

We can also plot a 3d histogram of the Gaussian samples:

In [ ]:
# Prepare the figure
fig = plt.figure(figsize = (3,3))
ax = fig.add_subplot(projection = '3d')

# Prepare the colormap
cmap = colormaps.get_cmap('inferno')
use_cmap = cmap(np.arange(cmap.N))
use_cmap[:,-1] = np.linspace(0,1,cmap.N)**1.7
use_cmap = colors.ListedColormap(use_cmap)

# Generate the uniform histogram
uniform_histogram(ax,pts_samples,GAUSSIAN_samples,bins = N_bins,filled = 0.3,cmap = use_cmap,verbose = False)

# Edit some figure settings and save it
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.zaxis.labelpad=-3.5
fig.savefig('Images/3_Histogram-Uniform_Gaussian.png',dpi = 300)

# Tree Plotting

Often, it is useful to have a dynamic bin-size in 3d histogram plotting because the number of polygons one must plot scales as $6N^3$. This is especially true in cases where (1) a small binsize is required and (2) most of the points of interest are within a small region. Dynamic sizing can be done using the `tree_histogram` method.

We can begin by importing the method:

In [ ]:
from cplt3d.generator_funcs import tree_histogram

Then generating a tree histogram is as easy as generating a regular histogram:

In [ ]:
# Prepare the figure
fig = plt.figure(figsize = (3,3))
ax = fig.add_subplot(projection = '3d')

# Plot the tree histogram
poly = tree_histogram(ax,pts_samples,GAUSSIAN_samples,cmap = use_cmap,
               filled=None,verbose = True,dist = 'sigmoid',
               min_resolution = None,max_resolution = None,edgecolor_function = lambda color:(0,0,0,0.01))

# Edit some figure settings and save it
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.zaxis.labelpad=-3.5
fig.savefig('Images/4_Histogram-Tree_Gaussian_1.png',dpi = 300)

As another example, we can try the method on the dual-Gaussian bimodal case:

In [ ]:
# Prepare the figure
fig = plt.figure(figsize = (6*2,6))
ax = fig.add_subplot(121,projection = '3d')
ax2 = fig.add_subplot(122,projection = '3d')

# Prepare the colormap and region to plot
cmap = colormaps.get_cmap('inferno')
use_cmap = cmap(np.arange(cmap.N))
use_cmap[:,-1] = np.linspace(0,1,cmap.N)**2
use_cmap = colors.ListedColormap(use_cmap)
d = 3

# Plot with edges colored
poly = tree_histogram(ax,np.array(pts_samples_bi),np.array(GAUSSIAN_samples_bi),cmap = use_cmap,
               filled=None,verbose = False,
               _range = [[-d,d],[-d,d],[-d,d]],
               min_resolution = 1,max_resolution = 5,
               vmin = 1,norm = LogNorm,edgecolor_function = lambda x:(0,0,0,0.2),linewidths = 0.4)

# Plot without edges colored
poly = tree_histogram(ax2,np.array(pts_samples_bi),np.array(GAUSSIAN_samples_bi),cmap = use_cmap,
               filled=None,verbose = False,
               _range = [[-d,d],[-d,d],[-d,d]],
               min_resolution = 1,max_resolution = 5,
               vmin = 1,norm = LogNorm)

# Edit some figure settings and save it
ax.set_xlim(-d,d)
ax.set_ylim(-d,d)
ax.set_zlim(-d,d)
fig.tight_layout()
fig.savefig('Images/6_Histogram-Tree Dual-Gaussian.png',dpi = 300)

# Rotating Plots

`cplt3d` also makes it easy to animate rotating 3d plots. The process is parallelized which makes it substantially faster for more complex plots:

In [ ]:
if animate and __name__ == '__main__':
    folder_in = './Images'
    spin_3d_plot(fig,[ax,ax2],'Images/7_Gaussian_Histogram',step=1,merge=True,delete=True,fps = 15,
                parallel = True,verbose = True,Animation_Generation_Folder=folder_in,dpi = 300)